In [1]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']

for intent in intents['intents']:
    try:
        for pattern in intent['patterns']:
            wordList = nltk.word_tokenize(pattern)
            words.extend(wordList)
            documents.append((wordList, intent['tag']))
            if intent['tag'] not in classes:
                classes.append(intent['tag'])
    except Exception as ex:
        print(intent, ">>", ex)

words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
outputEmpty = [0] * len(classes)

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]


model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(trainX), np.array(trainY), epochs=40, batch_size=5, verbose=1)
model.save('bot_model.h5', hist)
print('Done')


Epoch 1/40
81/81 [==============================] - 11s 8ms/step - loss: 3.6041 - accuracy: 0.0494
Epoch 2/40
81/81 [==============================] - 0s 2ms/step - loss: 3.4290 - accuracy: 0.1185
Epoch 3/40
81/81 [==============================] - 0s 2ms/step - loss: 3.1804 - accuracy: 0.1877
Epoch 4/40
81/81 [==============================] - 0s 3ms/step - loss: 2.9093 - accuracy: 0.2395
Epoch 5/40
81/81 [==============================] - 0s 2ms/step - loss: 2.6791 - accuracy: 0.3062
Epoch 6/40
81/81 [==============================] - 0s 2ms/step - loss: 2.3908 - accuracy: 0.3654
Epoch 7/40
81/81 [==============================] - 0s 3ms/step - loss: 2.1940 - accuracy: 0.3827
Epoch 8/40
81/81 [==============================] - 0s 2ms/step - loss: 2.0375 - accuracy: 0.4346
Epoch 9/40
81/81 [==============================] - 0s 3ms/step - loss: 1.7941 - accuracy: 0.5259
Epoch 10/40
81/81 [==============================] - 0s 2ms/step - loss: 1.7155 - accuracy: 0.5136
Epoch 11/40
81/81 

D:\Anaconda\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Done
